In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.utils import shuffle
import sklearn 
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
import random
import warnings
warnings.filterwarnings('ignore')




In [4]:
# Total = np.unique(pd.read_csv("./188_wheeze.txt", header=None)[0].values)
Total = np.unique(Breath_dataframe_0.iloc[:,73].values)
name = []
for i in Total:
    name.append(i[:4])
unique = []
before_after = []
for i in Total:
    if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
        unique.append(i)
    else:
        before_after.append(i) 

Asthmatic_Female = []
Asthmatic_Male = []
Healthy_Male = []
Healthy_Female = []
NAsthma = []
before = {"male" : [], "female" : []}
after = {"male" : [], "female" : []}
for file in unique:
    if file.split("_")[2] == 'Asthma' or file.split("_")[2] == 'asthma' :
        if file.find("_M_")!=-1:
            Asthmatic_Male.append(file)
        if file.find("_F_")!=-1:
            Asthmatic_Female.append(file)
    if file.split("_")[2] == 'NAsthma':
        NAsthma.append(file)
    if file.find("_C_")!=-1:
        if file.find("_M_")!=-1:
            Healthy_Male.append(file)
        if file.find("_F_")!=-1:
            Healthy_Female.append(file)
            
for file in before_after:
        if file.find("efore") != -1:
            if file.find("_M_") != -1 or file.find("_m_")!=-1:
                before["male"].append(file)
            if file.find("_F_")!=-1:
                before["female"].append(file)
        if file.find("fter")!= -1:
            if file.find("_M_") != -1 or file.find("_m_")!=-1:
                after["male"].append(file)
            if file.find("_F_")!=-1:
                after["female"].append(file)

In [5]:
print(f"Total = {len(Total)}")
print(f"Asthmatic_male = {len(Asthmatic_Male)+ len(before['male'])+len(after['male'])}")
print(f"Asthmatic_female = {len(Asthmatic_Female)+ len(before['female'])+len(after['female'])}")
print(f"Healthy_Male = {len(Healthy_Male)}")
print(f"Healthy_female = {len(Healthy_Female)}")
print(f"Asthmatic_male_unique = {len(Asthmatic_Male)}")
print(f"Asthmatic_female_unique = {len(Asthmatic_Female)}")
print(f"Before_after = {len(before_after)}")
print(f"Before_male = {len(before['male'])}")
print(f"Before_female = {len(before['female'])}")
print(f"after_male = {len(after['male'])}")
print(f"after_female = {len(after['female'])}")

print(f"NAsthma = {len(NAsthma)}")

Total = 274
Asthmatic_male = 46
Asthmatic_female = 28
Healthy_Male = 55
Healthy_female = 51
Before_after = 84
NAsthma = 10


In [49]:
#Machine Learning Algorithm (MLA) Selection and Initialization
def MLA_selection(sound_dataframe, sound, path):    
    MLA = [
        #Ensemble Methods
#         ensemble.AdaBoostClassifier(),
#         ensemble.BaggingClassifier(),
#         ensemble.ExtraTreesClassifier(),
#         ensemble.GradientBoostingClassifier(),
        ensemble.RandomForestClassifier(),

        #Gaussian Processes
#         gaussian_process.GaussianProcessClassifier(),

        #GLM
#         linear_model.LogisticRegressionCV(),
#         linear_model.PassiveAggressiveClassifier(),
#         linear_model.RidgeClassifierCV(),
#         linear_model.SGDClassifier(),
#         linear_model.Perceptron(),

        #Navies Bayes
#         naive_bayes.BernoulliNB(),
#         naive_bayes.GaussianNB(),

        #Nearest Neighbor
#         neighbors.KNeighborsClassifier(),

        #SVM
        svm.SVC(probability=True),
#         svm.NuSVC(probability=True),
        svm.LinearSVC(),

        #Trees    
#         tree.DecisionTreeClassifier(),
# #         tree.ExtraTreeClassifier(),

        #Discriminant Analysis
#         discriminant_analysis.LinearDiscriminantAnalysis(),
#         discriminant_analysis.QuadraticDiscriminantAnalysis(),


#         XGBClassifier(max_depth = 8,
#                      subsample = 0.8,
#                      learning_rate = 0.01,
#                      n_estimators = 450,
#                      min_child_weight = 1)
        XGBClassifier()
        ]


    MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy','MLA Train Accuracy Mean', 'MLA Test Accuracy', 'MLA Test Accuracy Mean','MLA Test Accuracy Std' ]
    MLA_compare = pd.DataFrame(columns = MLA_columns)
    row_index = 0
    for alg in MLA:
        MLA_name = alg.__class__.__name__
        MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
        MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
        test = []
        train = []
        for i in tqdm(range(0,10)):
            train_csv, test_csv = pd.read_csv(path+'/train'+sound+str(i)+'.csv'), pd.read_csv(path+'./test'+sound+str(i)+'.csv')
            X_train = train_csv.iloc[:,:sound_dataframe.shape[1]-2]
            y_train = train_csv.iloc[:,sound_dataframe.shape[1]-2]
            X_test = test_csv.iloc[:,:sound_dataframe.shape[1]-2]
            y_test = test_csv.iloc[:,sound_dataframe.shape[1]-2]
            #Scaling
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            X_train = pd.DataFrame(X_train)
            X_test = pd.DataFrame(X_test)

            alg.fit(X_train,y_train)
            test.append(metrics.accuracy_score(y_test,alg.predict(X_test)))

            train.append(metrics.accuracy_score(y_train,alg.predict(X_train)))
            
#             if alg.__class__.__name__ == 'XGBClassifier':
#                 filename = './CMN/XGB_model'+ str(i) + '.model'
#                 pickle.dump(alg, open(filename, 'wb'))
#                 train_csv.to_csv('./CMN/Train_XGB'+str(i)+'.csv', index=False)
                
        
        MLA_compare.loc[row_index, 'MLA Train Accuracy'] = train
        MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = np.mean(train)
        MLA_compare.loc[row_index, 'MLA Test Accuracy'] = test
        MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = np.mean(test)
        MLA_compare.loc[row_index, 'MLA Test Accuracy Std'] = np.std(test)


        row_index+=1

    
    MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
    
    return MLA_compare

In [6]:
def make_legit_sets(sound_dataframe, sound, path):
    for i in range(0,10):
        Train = np.loadtxt(f"./{path}/Train/Train_{i}.txt", dtype= 'str')
        Test = np.loadtxt(f"./{path}/Test/Test_{i}.txt", dtype= 'str')


        test_dataframe = pd.DataFrame()
        for j, line in enumerate(Test):
            A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Test[j]))]
            test_dataframe = pd.DataFrame.append(test_dataframe,A)

        train_dataframe = pd.DataFrame()
        for j, line in enumerate(Train):
            A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Train[j]))]
            train_dataframe = pd.DataFrame.append(train_dataframe,A)
#################################################
        train_dataframe.to_csv('./breathe_set_csv_exp_6/train'+sound+str(i)+'.csv', index=False)
        test_dataframe.to_csv('./breathe_set_csv_exp_6/test'+sound+str(i)+'.csv', index=False)
    return None

In [6]:
Wheeze_dataframe = pd.read_csv("./breath_csv/Wheeze.csv")
Wheeze_dataframe_1 = pd.read_csv("./breath_csv/Wheeze_1.csv")
Wheeze_dataframe_2 = pd.read_csv("./breath_csv/Wheeze_2.csv")
Wheeze_dataframe_3 = pd.read_csv("./breath_csv/Wheeze_3.csv")

In [9]:
make_legit_sets(Wheeze_dataframe, 'Wheeze', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_1, 'Wheeze_1', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_2, 'Wheeze_2', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_3, 'Wheeze_3', "Wheeze_sets")

In [75]:
MLA_Wheeze = MLA_selection(Wheeze_dataframe, 'Wheeze', "./breathe_set_csv_exp_4/")
MLA_Wheeze_1 = MLA_selection(Wheeze_dataframe_1, 'Wheeze_1', "./breathe_set_csv_exp_4/")
MLA_Wheeze_2 = MLA_selection(Wheeze_dataframe_2, 'Wheeze_2', "./breathe_set_csv_exp_4/")
MLA_Wheeze_3 = MLA_selection(Wheeze_dataframe_3, 'Wheeze_3', "./breathe_set_csv_exp_4/")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[06:19:37] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:19:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:19:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:19:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[06:20:53] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:20:53] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:20:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:20:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [76]:
MLA_Wheeze

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9061224489795918, 0.8445378151260504, 0.890...",0.862794,0.0401354
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9142857142857143, 0.8697478991596639, 0.898...",0.857496,0.0493695
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9721577726218097, 0.97, 0.9664586583463338,...",0.972642,"[0.8979591836734694, 0.8151260504201681, 0.843...",0.815662,0.0482381
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8894044856921887, 0.9023076923076923, 0.883...",0.898692,"[0.8938775510204081, 0.7521008403361344, 0.839...",0.807796,0.0517178


In [77]:
MLA_Wheeze_1

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8571428571428571, 0.7647058823529411, 0.828...",0.784965,0.045702
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8285714285714286, 0.7815126050420168, 0.824...",0.78456,0.0395398
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8507347254447022, 0.8584615384615385, 0.851...",0.861377,"[0.8244897959183674, 0.7521008403361344, 0.859...",0.782321,0.0444023
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9303944315545244, 0.9276923076923077, 0.926...",0.932318,"[0.8081632653061225, 0.7563025210084033, 0.816...",0.779208,0.0401786


In [78]:
MLA_Wheeze_2

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9346938775510204, 0.8361344537815126, 0.875...",0.851052,0.0435679
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9346938775510204, 0.8529411764705882, 0.875...",0.845398,0.0509733
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9621036349574633, 0.9607692307692308, 0.961...",0.966538,"[0.926530612244898, 0.8529411764705882, 0.8164...",0.837902,0.0519563
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.888631090487239, 0.8923076923076924, 0.8923...",0.891199,"[0.889795918367347, 0.7605042016806722, 0.8125...",0.806633,0.0477326


In [79]:
MLA_Wheeze_3

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8646558391337974, 0.8784615384615385, 0.867...",0.872846,"[0.8857142857142857, 0.8025210084033614, 0.828...",0.803741,0.0384212
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9450889404485692, 0.946923076923077, 0.9446...",0.946774,"[0.889795918367347, 0.7983193277310925, 0.7812...",0.798646,0.0367506
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8734693877551021, 0.7773109243697479, 0.824...",0.794951,0.0451413
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8408163265306122, 0.773109243697479, 0.8320...",0.792802,0.0358028


In [60]:
Wheeze_dataframe_0 = pd.read_csv("./breath_csv/Wheeze_0.csv")
Wheeze_dataframe_5 = pd.read_csv("./breath_csv/Wheeze_5.csv")
Wheeze_dataframe_10 = pd.read_csv("./breath_csv/Wheeze_10.csv")
Wheeze_dataframe_15 = pd.read_csv("./breath_csv/Wheeze_15.csv")
Wheeze_dataframe_20 = pd.read_csv("./breath_csv/Wheeze_20.csv")
Wheeze_dataframe_25 = pd.read_csv("./breath_csv/Wheeze_25.csv")
Wheeze_dataframe_30 = pd.read_csv("./breath_csv/Wheeze_30.csv")
Wheeze_dataframe_35 = pd.read_csv("./breath_csv/Wheeze_35.csv")
Wheeze_dataframe_40 = pd.read_csv("./breath_csv/Wheeze_40.csv")
Wheeze_dataframe_45 = pd.read_csv("./breath_csv/Wheeze_45.csv")
Wheeze_dataframe_50 = pd.read_csv("./breath_csv/Wheeze_50.csv")

In [61]:
make_legit_sets(Wheeze_dataframe_0, 'Wheeze_0', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_5, 'Wheeze_5', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_10, 'Wheeze_10', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_15, 'Wheeze_15', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_20, 'Wheeze_20', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_25, 'Wheeze_25', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_30, 'Wheeze_30', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_35, 'Wheeze_35', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_40, 'Wheeze_40', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_45, 'Wheeze_45', "Wheeze_sets")
make_legit_sets(Wheeze_dataframe_50, 'Wheeze_50', "Wheeze_sets")

In [62]:
MLA_wheeze_0 = MLA_selection(Wheeze_dataframe_0, 'Wheeze_0', "./breathe_set_csv_exp_4/")
MLA_wheeze_5 = MLA_selection(Wheeze_dataframe_5, 'Wheeze_5', "./breathe_set_csv_exp_4/")
MLA_wheeze_10 = MLA_selection(Wheeze_dataframe_10, 'Wheeze_10', "./breathe_set_csv_exp_4/")
MLA_wheeze_15 = MLA_selection(Wheeze_dataframe_15, 'Wheeze_15', "./breathe_set_csv_exp_4/")
MLA_wheeze_20 = MLA_selection(Wheeze_dataframe_20, 'Wheeze_20', "./breathe_set_csv_exp_4/")
MLA_wheeze_25 = MLA_selection(Wheeze_dataframe_25, 'Wheeze_25', "./breathe_set_csv_exp_4/")
MLA_wheeze_30 = MLA_selection(Wheeze_dataframe_30, 'Wheeze_30', "./breathe_set_csv_exp_4/")
MLA_wheeze_35 = MLA_selection(Wheeze_dataframe_35, 'Wheeze_35', "./breathe_set_csv_exp_4/")
MLA_wheeze_40 = MLA_selection(Wheeze_dataframe_40, 'Wheeze_40', "./breathe_set_csv_exp_4/")
MLA_wheeze_45 = MLA_selection(Wheeze_dataframe_45, 'Wheeze_45', "./breathe_set_csv_exp_4/")
MLA_wheeze_50 = MLA_selection(Wheeze_dataframe_50, 'Wheeze_50', "./breathe_set_csv_exp_4/")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[05:50:03] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:50:08] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:50:18] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:50:23] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[05:58:55] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:58:58] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:59:02] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:59:05] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [63]:
MLA_wheeze_0

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8979591836734694, 0.8067226890756303, 0.878...",0.825883,0.0498517
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8979591836734694, 0.7941176470588235, 0.867...",0.824202,0.0438369
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9450889404485692, 0.9461538461538461, 0.944...",0.95146,"[0.8857142857142857, 0.8025210084033614, 0.820...",0.811478,0.0490403
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.877030162412993, 0.8838461538461538, 0.8728...",0.885553,"[0.9061224489795918, 0.7773109243697479, 0.859...",0.797247,0.0590706


In [64]:
MLA_wheeze_5

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9061224489795918, 0.7899159663865546, 0.851...",0.8227,0.052671
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9061224489795918, 0.8025210084033614, 0.859...",0.822147,0.0534777
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9466357308584686, 0.9484615384615385, 0.948...",0.951981,"[0.8816326530612245, 0.8277310924369747, 0.824...",0.811212,0.0552687
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8847641144624904, 0.89, 0.8806552262090483,...",0.893351,"[0.9102040816326531, 0.8109243697478992, 0.851...",0.808739,0.0588626


In [65]:
MLA_wheeze_10

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 0.9992125984251968, 1.0, 1.0, ...",0.999845,"[0.9183673469387755, 0.8067226890756303, 0.847...",0.826877,0.0522997
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.945862335653519, 0.9476923076923077, 0.9492...",0.951909,"[0.9061224489795918, 0.8613445378151261, 0.859...",0.825491,0.0520221
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9224489795918367, 0.7983193277310925, 0.875...",0.82279,0.0623617
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8863109048723898, 0.8838461538461538, 0.876...",0.888271,"[0.8857142857142857, 0.7352941176470589, 0.859...",0.79629,0.0592915


In [66]:
MLA_wheeze_15

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 0.9992125984251968, 1.0, 1.0, ...",0.999921,"[0.9346938775510204, 0.8319327731092437, 0.875...",0.842104,0.0498777
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9224489795918367, 0.8151260504201681, 0.855...",0.838738,0.0476872
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9559164733178654, 0.95, 0.9492979719188768,...",0.957191,"[0.9061224489795918, 0.8613445378151261, 0.859...",0.824198,0.0567637
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8847641144624904, 0.8923076923076924, 0.882...",0.892286,"[0.889795918367347, 0.7563025210084033, 0.8398...",0.808507,0.0559755


In [67]:
MLA_wheeze_20

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 0.9992125984251968, 1.0, 1.0, ...",0.999921,"[0.9306122448979591, 0.8487394957983193, 0.875...",0.845931,0.0498238
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9306122448979591, 0.8529411764705882, 0.875...",0.845395,0.0520998
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9551430781129157, 0.9507692307692308, 0.956...",0.960132,"[0.9224489795918367, 0.8697478991596639, 0.863...",0.834722,0.0555252
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8824439288476411, 0.8953846153846153, 0.884...",0.889604,"[0.9061224489795918, 0.7647058823529411, 0.843...",0.817977,0.0452526


In [68]:
MLA_wheeze_25

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9306122448979591, 0.8403361344537815, 0.867...",0.844822,0.0493536
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9346938775510204, 0.8445378151260504, 0.859...",0.842665,0.0478151
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9621036349574633, 0.9607692307692308, 0.961...",0.966538,"[0.926530612244898, 0.8529411764705882, 0.8164...",0.837902,0.0519563
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8870843000773395, 0.8915384615384615, 0.892...",0.891205,"[0.8857142857142857, 0.7563025210084033, 0.812...",0.804974,0.0477323


In [69]:
MLA_wheeze_30

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9387755102040817, 0.8445378151260504, 0.835...",0.84747,0.0465912
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9183673469387755, 0.819327731092437, 0.8476...",0.832815,0.0461284
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9597834493426141, 0.96, 0.9648985959438378,...",0.96709,"[0.926530612244898, 0.8151260504201681, 0.8398...",0.829502,0.047383
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8870843000773395, 0.8923076923076924, 0.886...",0.893852,"[0.9020408163265307, 0.7815126050420168, 0.789...",0.819318,0.0364086


In [70]:
MLA_wheeze_35

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9061224489795918, 0.8487394957983193, 0.820...",0.825188,0.0448311
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9597834493426141, 0.9646153846153847, 0.965...",0.965704,"[0.9061224489795918, 0.8277310924369747, 0.824...",0.823919,0.0418663
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9142857142857143, 0.8277310924369747, 0.816...",0.821434,0.0490248
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.888631090487239, 0.8869230769230769, 0.8822...",0.889634,"[0.8857142857142857, 0.7815126050420168, 0.777...",0.805244,0.0354808


In [71]:
MLA_wheeze_40

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9020408163265307, 0.8571428571428571, 0.820...",0.821839,0.0467198
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9613302397525135, 0.9607692307692308, 0.957...",0.963138,"[0.9020408163265307, 0.8151260504201681, 0.820...",0.817164,0.044771
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9992266047950503, 1.0, 0.999219968798752, 1...",0.999615,"[0.8979591836734694, 0.8403361344537815, 0.816...",0.814955,0.0507342
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8747099767981439, 0.8838461538461538, 0.875...",0.886102,"[0.8816326530612245, 0.7815126050420168, 0.761...",0.786197,0.0406657


In [72]:
MLA_wheeze_45

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.9020408163265307, 0.8361344537815126, 0.808...",0.819029,0.040269
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 0.999219968798752, 0.99921259842519...",0.999766,"[0.8816326530612245, 0.8361344537815126, 0.820...",0.805194,0.0512598
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8669760247486465, 0.8738461538461538, 0.870...",0.876507,"[0.889795918367347, 0.773109243697479, 0.76171...",0.794319,0.041029
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9566898685228151, 0.9592307692307692, 0.955...",0.962747,"[0.889795918367347, 0.7941176470588235, 0.8046...",0.792668,0.0481288


In [73]:
MLA_wheeze_50

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8734693877551021, 0.8319327731092437, 0.785...",0.80965,0.0375153
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9992307692307693, 0.999219968798752, 0...",0.999538,"[0.889795918367347, 0.8277310924369747, 0.8320...",0.804897,0.0535399
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8739365815931941, 0.8853846153846154, 0.872...",0.882987,"[0.8734693877551021, 0.7563025210084033, 0.777...",0.797445,0.0407303
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9566898685228151, 0.96, 0.9609984399375975,...",0.963401,"[0.8734693877551021, 0.8109243697478992, 0.796...",0.79547,0.0406523


In [4]:
breath_dataframe = pd.read_csv("./breath_csv_2//Breath.csv")
breath_dataframe_1 = pd.read_csv("./breath_csv_2//Breath_1.csv")
breath_dataframe_2 = pd.read_csv("./breath_csv_2//Breath_2.csv")
breath_dataframe_3 = pd.read_csv("./breath_csv_2//Breath_3.csv")

In [5]:
make_legit_sets(breath_dataframe, 'breath', "breath_sets")
make_legit_sets(breath_dataframe_1, 'breath_1', "breath_sets")
make_legit_sets(breath_dataframe_2, 'breath_2', "breath_sets")
make_legit_sets(breath_dataframe_3, 'breath_3', "breath_sets")

In [6]:
MLA_breath = MLA_selection(breath_dataframe, 'breath', "./breathe_set_csv_exp_5/")
MLA_breath_1 = MLA_selection(breath_dataframe_1, 'breath_1', "./breathe_set_csv_exp_5/")
MLA_breath_2 = MLA_selection(breath_dataframe_2, 'breath_2', "./breathe_set_csv_exp_5/")
MLA_breath_3 = MLA_selection(breath_dataframe_3, 'breath_3', "./breathe_set_csv_exp_5/")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[09:57:55] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:58:00] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:58:06] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:58:11] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[10:02:14] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:02:20] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:02:25] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:02:30] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [7]:
MLA_breath

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.812, 0.7213740458015268, 0.8185483870967742...",0.766754,0.0704297
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995222169135213, 1.0, 0.9990453460620525, ...",0.999569,"[0.828, 0.6870229007633588, 0.8346774193548387...",0.766674,0.0779409
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9627329192546584, 0.9639596347909659, 0.960...",0.961303,"[0.752, 0.6793893129770993, 0.7459677419354839...",0.733473,0.0657154
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8413760152890588, 0.8447861604997597, 0.833...",0.840872,"[0.744, 0.6564885496183206, 0.7661290322580645...",0.725567,0.0773419


In [8]:
MLA_breath_1

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.808, 0.7061068702290076, 0.6854838709677419...",0.730445,0.0686485
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9111323459149546, 0.9144641999038924, 0.912...",0.913944,"[0.772, 0.7137404580152672, 0.7016129032258065...",0.724779,0.0584042
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9995194617972128, 0.9995226730310263, ...",0.999331,"[0.768, 0.6793893129770993, 0.6814516129032258...",0.72346,0.062938
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.7916865742952699, 0.7972128784238347, 0.806...",0.795913,"[0.736, 0.648854961832061, 0.625, 0.7792207792...",0.696639,0.0755518


In [9]:
MLA_breath_2

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9995194617972128, 1.0, 1.0, 1.0, 1.0, ...",0.999952,"[0.812, 0.683206106870229, 0.8145161290322581,...",0.739855,0.0957061
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.82, 0.6946564885496184, 0.7903225806451613,...",0.739023,0.0929201
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9421882465360726, 0.9432964920711197, 0.936...",0.940972,"[0.788, 0.6984732824427481, 0.8306451612903226...",0.730691,0.0807408
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8275203057811753, 0.8399807784718886, 0.827...",0.833895,"[0.792, 0.6717557251908397, 0.7580645161290323...",0.730318,0.0861963


In [10]:
MLA_breath_3

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8279980888676541, 0.8370975492551658, 0.823...",0.829445,"[0.804, 0.683206106870229, 0.8266129032258065,...",0.777279,0.0766033
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9216435738174868, 0.9308024987986545, 0.921...",0.924285,"[0.768, 0.6870229007633588, 0.8185483870967742...",0.766023,0.0679336
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9990444338270426, 0.9990389235944258, 0.999...",0.99933,"[0.804, 0.6450381679389313, 0.7903225806451613...",0.74982,0.0824955
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.816, 0.6603053435114504, 0.8024193548387096...",0.748347,0.071622


Breath 10

In [19]:
for i in range(0,51,5):
    print(f"MLA_breath_{i} = MLA_selection(Breath_dataframe_{i}, 'breath_{i}', './breathe_set_csv_exp_5/') ")

MLA_breath_0 = MLA_selection(Breath_dataframe_0, 'breath_0', './breathe_set_csv_exp_5/') 
MLA_breath_5 = MLA_selection(Breath_dataframe_5, 'breath_5', './breathe_set_csv_exp_5/') 
MLA_breath_10 = MLA_selection(Breath_dataframe_10, 'breath_10', './breathe_set_csv_exp_5/') 
MLA_breath_15 = MLA_selection(Breath_dataframe_15, 'breath_15', './breathe_set_csv_exp_5/') 
MLA_breath_20 = MLA_selection(Breath_dataframe_20, 'breath_20', './breathe_set_csv_exp_5/') 
MLA_breath_25 = MLA_selection(Breath_dataframe_25, 'breath_25', './breathe_set_csv_exp_5/') 
MLA_breath_30 = MLA_selection(Breath_dataframe_30, 'breath_30', './breathe_set_csv_exp_5/') 
MLA_breath_35 = MLA_selection(Breath_dataframe_35, 'breath_35', './breathe_set_csv_exp_5/') 
MLA_breath_40 = MLA_selection(Breath_dataframe_40, 'breath_40', './breathe_set_csv_exp_5/') 
MLA_breath_45 = MLA_selection(Breath_dataframe_45, 'breath_45', './breathe_set_csv_exp_5/') 
MLA_breath_50 = MLA_selection(Breath_dataframe_50, 'breath_50', './breathe_s

In [2]:
Breath_dataframe_0 = pd.read_csv('./breath_csv_2/Breath_0.csv')
Breath_dataframe_5 = pd.read_csv('./breath_csv_2/Breath_5.csv')
Breath_dataframe_10 = pd.read_csv('./breath_csv_2/Breath_10.csv')
Breath_dataframe_15 = pd.read_csv('./breath_csv_2/Breath_15.csv')
Breath_dataframe_20 = pd.read_csv('./breath_csv_2/Breath_20.csv')
Breath_dataframe_25 = pd.read_csv('./breath_csv_2/Breath_25.csv')
Breath_dataframe_30 = pd.read_csv('./breath_csv_2/Breath_30.csv')
Breath_dataframe_35 = pd.read_csv('./breath_csv_2/Breath_35.csv')
Breath_dataframe_40 = pd.read_csv('./breath_csv_2/Breath_40.csv')
Breath_dataframe_45 = pd.read_csv('./breath_csv_2/Breath_45.csv')
Breath_dataframe_50 = pd.read_csv('./breath_csv_2/Breath_50.csv')


In [16]:
make_legit_sets(Breath_dataframe_0, 'breath_0', 'breath_sets')
make_legit_sets(Breath_dataframe_5, 'breath_5', 'breath_sets')
make_legit_sets(Breath_dataframe_10, 'breath_10', 'breath_sets')
make_legit_sets(Breath_dataframe_15, 'breath_15', 'breath_sets')
make_legit_sets(Breath_dataframe_20, 'breath_20', 'breath_sets')
make_legit_sets(Breath_dataframe_25, 'breath_25', 'breath_sets')
make_legit_sets(Breath_dataframe_30, 'breath_30', 'breath_sets')
make_legit_sets(Breath_dataframe_35, 'breath_35', 'breath_sets')
make_legit_sets(Breath_dataframe_40, 'breath_40', 'breath_sets')
make_legit_sets(Breath_dataframe_45, 'breath_45', 'breath_sets')
make_legit_sets(Breath_dataframe_50, 'breath_50', 'breath_sets')

In [6]:
MLA_breath_0 = MLA_selection(Breath_dataframe_0, 'breath_0', './breathe_set_csv_exp_5/') 
MLA_breath_5 = MLA_selection(Breath_dataframe_5, 'breath_5', './breathe_set_csv_exp_5/') 
MLA_breath_10 = MLA_selection(Breath_dataframe_10, 'breath_10', './breathe_set_csv_exp_5/') 
MLA_breath_15 = MLA_selection(Breath_dataframe_15, 'breath_15', './breathe_set_csv_exp_5/') 
MLA_breath_20 = MLA_selection(Breath_dataframe_20, 'breath_20', './breathe_set_csv_exp_5/') 
MLA_breath_25 = MLA_selection(Breath_dataframe_25, 'breath_25', './breathe_set_csv_exp_5/') 
MLA_breath_30 = MLA_selection(Breath_dataframe_30, 'breath_30', './breathe_set_csv_exp_5/') 
MLA_breath_35 = MLA_selection(Breath_dataframe_35, 'breath_35', './breathe_set_csv_exp_5/') 
MLA_breath_40 = MLA_selection(Breath_dataframe_40, 'breath_40', './breathe_set_csv_exp_5/') 
MLA_breath_45 = MLA_selection(Breath_dataframe_45, 'breath_45', './breathe_set_csv_exp_5/') 
MLA_breath_50 = MLA_selection(Breath_dataframe_50, 'breath_50', './breathe_set_csv_exp_5/') 


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[13:58:26] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:58:33] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:58:40] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:58:47] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[14:17:39] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:17:46] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:17:52] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:17:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [7]:
MLA_breath_0

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9995194617972128, 1.0, 1.0, 1.0, 0.999...",0.999857,"[0.8, 0.7061068702290076, 0.8024193548387096, ...",0.752204,0.0750981
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.792, 0.7213740458015268, 0.7540322580645161...",0.745117,0.0674417
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.924988055422838, 0.9298414223930802, 0.9231...",0.927585,"[0.784, 0.6908396946564885, 0.7620967741935484...",0.725607,0.0670972
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8193979933110368, 0.8164344065353195, 0.806...",0.81448,"[0.74, 0.6526717557251909, 0.6895161290322581,...",0.692194,0.0810505


In [8]:
MLA_breath_5

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995222169135213, 0.9995194617972128, 1.0, ...",0.999569,"[0.792, 0.7099236641221374, 0.8225806451612904...",0.749377,0.0779257
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.784, 0.7251908396946565, 0.7782258064516129...",0.747547,0.0672426
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9216435738174868, 0.932724651609803, 0.9212...",0.927014,"[0.772, 0.6679389312977099, 0.7580645161290323...",0.712978,0.0737255
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8217869087434305, 0.8130706391158097, 0.805...",0.816298,"[0.756, 0.6679389312977099, 0.7379032258064516...",0.696875,0.0844644


In [9]:
MLA_breath_10

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.784, 0.7175572519083969, 0.7983870967741935...",0.742004,0.0793638
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995222169135213, 0.9995194617972128, 1.0, ...",0.999809,"[0.732, 0.7099236641221374, 0.8185483870967742...",0.738188,0.0835424
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9273769708552317, 0.9346468044209515, 0.925...",0.931366,"[0.764, 0.683206106870229, 0.7338709677419355,...",0.71115,0.0677753
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8184424271380794, 0.8207592503604036, 0.808...",0.819988,"[0.744, 0.6755725190839694, 0.717741935483871,...",0.693253,0.088087


In [10]:
MLA_breath_15

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.796, 0.7290076335877863, 0.8064516129032258...",0.744487,0.0868747
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995222169135213, 1.0, 1.0, 1.0, 1.0, 0.999...",0.999856,"[0.78, 0.7175572519083969, 0.8387096774193549,...",0.742018,0.087782
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9345437171524128, 0.9375300336376742, 0.930...",0.93472,"[0.776, 0.6641221374045801, 0.7661290322580645...",0.72147,0.0708638
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8198757763975155, 0.8241230177799135, 0.808...",0.822711,"[0.748, 0.6564885496183206, 0.7540322580645161...",0.699132,0.0841056


In [11]:
MLA_breath_20

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.792, 0.7175572519083969, 0.8185483870967742...",0.74363,0.0910904
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.804, 0.7213740458015268, 0.8185483870967742...",0.740879,0.0907456
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9426660296225514, 0.9413743392599712, 0.936...",0.939921,"[0.78, 0.7099236641221374, 0.7943548387096774,...",0.730458,0.0737238
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8284758719541329, 0.8366170110523786, 0.809...",0.828741,"[0.76, 0.6679389312977099, 0.7701612903225806,...",0.716482,0.088979


In [12]:
MLA_breath_25

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9995194617972128, 1.0, 1.0, 1.0, 1.0, ...",0.999952,"[0.812, 0.683206106870229, 0.8145161290322581,...",0.739855,0.0957061
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.804, 0.683206106870229, 0.8024193548387096,...",0.734816,0.0856085
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9421882465360726, 0.9432964920711197, 0.936...",0.940972,"[0.788, 0.6984732824427481, 0.8306451612903226...",0.730691,0.0807408
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8275203057811753, 0.8399807784718886, 0.825...",0.833277,"[0.792, 0.6717557251908397, 0.7580645161290323...",0.730469,0.088623


In [13]:
MLA_breath_30

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.816, 0.6946564885496184, 0.8064516129032258...",0.746336,0.0879082
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9995194617972128, 0.9995226730310263, ...",0.999904,"[0.792, 0.6755725190839694, 0.8024193548387096...",0.746075,0.0942234
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.94696607740086, 0.9442575684766938, 0.94224...",0.943172,"[0.784, 0.7022900763358778, 0.8266129032258065...",0.734008,0.0808948
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8322981366459627, 0.8457472369053339, 0.825...",0.836864,"[0.76, 0.648854961832061, 0.7782258064516129, ...",0.73039,0.0899813


In [14]:
MLA_breath_35

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9995222169135213, 0.9995194617972128, 0.999...",0.999568,"[0.788, 0.6870229007633588, 0.7741935483870968...",0.742609,0.0853906
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8, 0.6679389312977099, 0.8145161290322581, ...",0.736114,0.0939631
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9474438604873388, 0.9471407976934166, 0.943...",0.942603,"[0.756, 0.6526717557251909, 0.8064516129032258...",0.728837,0.0790274
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8327759197324415, 0.8423834694858241, 0.824...",0.836532,"[0.74, 0.6412213740458015, 0.7862903225806451,...",0.72251,0.0827679


In [15]:
MLA_breath_40

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9995194617972128, 0.9990453460620525, ...",0.999568,"[0.804, 0.6755725190839694, 0.782258064516129,...",0.753695,0.0880127
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.832, 0.6679389312977099, 0.8024193548387096...",0.748831,0.0871562
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9488772097467749, 0.948582412301778, 0.9441...",0.944666,"[0.76, 0.6793893129770993, 0.8266129032258065,...",0.739708,0.0755281
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8256091734352604, 0.8385391638635271, 0.820...",0.828835,"[0.748, 0.648854961832061, 0.7983870967741935,...",0.721248,0.0872215


In [16]:
MLA_breath_45

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9990444338270426, 0.9995194617972128, 0.998...",0.999234,"[0.796, 0.6450381679389313, 0.8064516129032258...",0.749028,0.097042
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8, 0.6603053435114504, 0.8024193548387096, ...",0.747322,0.0741989
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8198757763975155, 0.8395002402691014, 0.820...",0.831349,"[0.784, 0.6374045801526718, 0.7862903225806451...",0.732425,0.086569
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9507883420926899, 0.952426717924075, 0.9451...",0.946103,"[0.768, 0.6412213740458015, 0.7782258064516129...",0.726897,0.0617964


In [17]:
MLA_breath_50

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.9990389235944258, 0.9990453460620525, ...",0.999474,"[0.82, 0.6641221374045801, 0.8185483870967742,...",0.756984,0.0988959
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.804, 0.6679389312977099, 0.8024193548387096...",0.751713,0.0789307
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8303870043000477, 0.8515136953387794, 0.829...",0.838003,"[0.772, 0.6259541984732825, 0.7983870967741935...",0.744417,0.0837044
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9498327759197325, 0.9490629505045651, 0.949...",0.947532,"[0.76, 0.6450381679389313, 0.782258064516129, ...",0.731446,0.064504


### EXP_7

In [34]:
breath_dataframe = pd.read_csv("./breath_csv_2//Breath.csv")
breath_dataframe_1 = pd.read_csv("./breath_csv_2//Breath_1.csv")
breath_dataframe_2 = pd.read_csv("./breath_csv_2//Breath_2.csv")
breath_dataframe_3 = pd.read_csv("./breath_csv_2//Breath_3.csv")

In [14]:
make_legit_sets(breath_dataframe, 'breath', "breath_sets_wo_after/")
make_legit_sets(breath_dataframe_1, 'breath_1', "breath_sets_wo_after/")
make_legit_sets(breath_dataframe_2, 'breath_2', "breath_sets_wo_after/")
make_legit_sets(breath_dataframe_3, 'breath_3', "breath_sets_wo_after/")

In [15]:
MLA_breath = MLA_selection(breath_dataframe, 'breath', "./breathe_set_csv_exp_6/")
MLA_breath_1 = MLA_selection(breath_dataframe_1, 'breath_1', "./breathe_set_csv_exp_6/")
MLA_breath_2 = MLA_selection(breath_dataframe_2, 'breath_2', "./breathe_set_csv_exp_6/")
MLA_breath_3 = MLA_selection(breath_dataframe_3, 'breath_3', "./breathe_set_csv_exp_6/")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[15:21:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:22:04] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:22:16] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:22:23] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[15:25:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:26:04] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:26:09] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:26:15] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [16]:
MLA_breath

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.999195494770716, 1.0, 1.0, 0.999196787...",0.999758,"[0.8382978723404255, 0.8264462809917356, 0.816...",0.80323,0.0411474
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8085106382978723, 0.8140495867768595, 0.8, ...",0.778312,0.0508312
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9664, 0.9646017699115044, 0.962248995983935...",0.965233,"[0.8, 0.7892561983471075, 0.7833333333333333, ...",0.771624,0.0340177
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8648, 0.8809332260659695, 0.888353413654618...",0.87654,"[0.7957446808510639, 0.7231404958677686, 0.8, ...",0.764033,0.0444302


In [17]:
MLA_breath_1

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7872340425531915, 0.7603305785123967, 0.766...",0.761891,0.0515541
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.999195494770716, 0.9991967871485944, 1...",0.999839,"[0.7574468085106383, 0.7603305785123967, 0.754...",0.757297,0.0427564
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.924, 0.9219629927594529, 0.9196787148594378...",0.92481,"[0.7702127659574468, 0.7272727272727273, 0.7, ...",0.726024,0.0402972
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8328, 0.8455349959774738, 0.836947791164658...",0.84131,"[0.7702127659574468, 0.6942148760330579, 0.716...",0.720538,0.0398731


In [18]:
MLA_breath_2

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8595744680851064, 0.7975206611570248, 0.85,...",0.805674,0.0461046
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8728, 0.8873692679002414, 0.888353413654618...",0.880338,"[0.825531914893617, 0.7892561983471075, 0.8166...",0.795559,0.0435088
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9504, 0.9549477071600966, 0.955020080321285...",0.955213,"[0.7957446808510639, 0.8099173553719008, 0.837...",0.790484,0.0436953
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8425531914893617, 0.768595041322314, 0.825,...",0.788838,0.0450013


In [19]:
MLA_breath_3

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.999195494770716, 1.0, 1.0, 0.999196787...",0.999676,"[0.7872340425531915, 0.7148760330578512, 0.758...",0.759291,0.0440705
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9272, 0.9372485921158488, 0.928514056224899...",0.932995,"[0.8085106382978723, 0.743801652892562, 0.7708...",0.758349,0.037612
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8042553191489362, 0.71900826446281, 0.73333...",0.746854,0.0485041
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8464, 0.8696701528559936, 0.849799196787148...",0.855625,"[0.7787234042553192, 0.6983471074380165, 0.729...",0.736054,0.0480124


In [20]:
Breath_dataframe_0 = pd.read_csv('./breath_csv_2/Breath_0.csv')
Breath_dataframe_5 = pd.read_csv('./breath_csv_2/Breath_5.csv')
Breath_dataframe_10 = pd.read_csv('./breath_csv_2/Breath_10.csv')
Breath_dataframe_15 = pd.read_csv('./breath_csv_2/Breath_15.csv')
Breath_dataframe_20 = pd.read_csv('./breath_csv_2/Breath_20.csv')
Breath_dataframe_25 = pd.read_csv('./breath_csv_2/Breath_25.csv')
Breath_dataframe_30 = pd.read_csv('./breath_csv_2/Breath_30.csv')
Breath_dataframe_35 = pd.read_csv('./breath_csv_2/Breath_35.csv')
Breath_dataframe_40 = pd.read_csv('./breath_csv_2/Breath_40.csv')
Breath_dataframe_45 = pd.read_csv('./breath_csv_2/Breath_45.csv')
Breath_dataframe_50 = pd.read_csv('./breath_csv_2/Breath_50.csv')

In [21]:
make_legit_sets(Breath_dataframe_0, 'breath_0', 'breath_sets_wo_after')
make_legit_sets(Breath_dataframe_5, 'breath_5', 'breath_sets_wo_after')
make_legit_sets(Breath_dataframe_10, 'breath_10', 'breath_sets_wo_after')
make_legit_sets(Breath_dataframe_15, 'breath_15', 'breath_sets_wo_after')
make_legit_sets(Breath_dataframe_20, 'breath_20', 'breath_sets_wo_after')
make_legit_sets(Breath_dataframe_25, 'breath_25', 'breath_sets_wo_after')
make_legit_sets(Breath_dataframe_30, 'breath_30', 'breath_sets_wo_after')
make_legit_sets(Breath_dataframe_35, 'breath_35', 'breath_sets_wo_after')
make_legit_sets(Breath_dataframe_40, 'breath_40', 'breath_sets_wo_after')
make_legit_sets(Breath_dataframe_45, 'breath_45', 'breath_sets_wo_after')
make_legit_sets(Breath_dataframe_50, 'breath_50', 'breath_sets_wo_after')

In [22]:
MLA_breath_0 = MLA_selection(Breath_dataframe_0, 'breath_0', './breathe_set_csv_exp_6/') 
MLA_breath_5 = MLA_selection(Breath_dataframe_5, 'breath_5', './breathe_set_csv_exp_6/') 
MLA_breath_10 = MLA_selection(Breath_dataframe_10, 'breath_10', './breathe_set_csv_exp_6/') 
MLA_breath_15 = MLA_selection(Breath_dataframe_15, 'breath_15', './breathe_set_csv_exp_6/') 
MLA_breath_20 = MLA_selection(Breath_dataframe_20, 'breath_20', './breathe_set_csv_exp_6/') 
MLA_breath_25 = MLA_selection(Breath_dataframe_25, 'breath_25', './breathe_set_csv_exp_6/') 
MLA_breath_30 = MLA_selection(Breath_dataframe_30, 'breath_30', './breathe_set_csv_exp_6/') 
MLA_breath_35 = MLA_selection(Breath_dataframe_35, 'breath_35', './breathe_set_csv_exp_6/') 
MLA_breath_40 = MLA_selection(Breath_dataframe_40, 'breath_40', './breathe_set_csv_exp_6/') 
MLA_breath_45 = MLA_selection(Breath_dataframe_45, 'breath_45', './breathe_set_csv_exp_6/') 
MLA_breath_50 = MLA_selection(Breath_dataframe_50, 'breath_50', './breathe_set_csv_exp_6/') 


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[06:53:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:53:58] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:54:01] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[06:54:05] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[07:03:25] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[07:03:28] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[07:03:31] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[07:03:33] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [23]:
MLA_breath_0

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 0.9991783073130649, 1.0, 1.0, ...",0.999918,"[0.8, 0.7768595041322314, 0.7791666666666667, ...",0.778626,0.0330421
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8, 0.7892561983471075, 0.8, 0.8208955223880...",0.774092,0.0387003
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8512, 0.8624296057924377, 0.853012048192771...",0.858864,"[0.8297872340425532, 0.71900826446281, 0.77083...",0.756159,0.0519835
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9408, 0.9388576025744167, 0.938955823293172...",0.941879,"[0.8425531914893617, 0.7520661157024794, 0.733...",0.756126,0.0479709


In [24]:
MLA_breath_5

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99919289...",0.999919,"[0.8425531914893617, 0.78099173553719, 0.8, 0....",0.791944,0.038769
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8085106382978723, 0.7727272727272727, 0.816...",0.786147,0.0449221
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9464, 0.9493161705551086, 0.944578313253012...",0.949473,"[0.8680851063829788, 0.768595041322314, 0.7708...",0.776768,0.0533862
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8688, 0.8656476267095736, 0.869076305220883...",0.87023,"[0.8468085106382979, 0.7603305785123967, 0.787...",0.764535,0.0477516


In [25]:
MLA_breath_10

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8680851063829788, 0.7892561983471075, 0.812...",0.79754,0.0462241
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8212765957446808, 0.8016528925619835, 0.812...",0.785553,0.0503161
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8688, 0.8761061946902655, 0.874698795180722...",0.878403,"[0.8340425531914893, 0.7603305785123967, 0.791...",0.779073,0.0450629
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9504, 0.9493161705551086, 0.950200803212851...",0.952385,"[0.851063829787234, 0.768595041322314, 0.7875,...",0.77753,0.0489476


In [26]:
MLA_breath_15

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.825531914893617, 0.8264462809917356, 0.8208...",0.799652,0.0471489
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9568, 0.9549477071600966, 0.951807228915662...",0.95496,"[0.8340425531914893, 0.8016528925619835, 0.820...",0.790567,0.0429215
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8170212765957446, 0.8140495867768595, 0.816...",0.78941,0.0484772
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.88, 0.8841512469831054, 0.8755020080321285,...",0.880013,"[0.8212765957446808, 0.768595041322314, 0.8, 0...",0.785551,0.0394611


In [27]:
MLA_breath_20

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8468085106382979, 0.8223140495867769, 0.845...",0.803848,0.0517677
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9504, 0.9509251810136766, 0.951807228915662...",0.95424,"[0.8212765957446808, 0.7975206611570248, 0.837...",0.791551,0.0459769
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8728, 0.8817377312952535, 0.884337349397590...",0.877741,"[0.825531914893617, 0.7768595041322314, 0.8125...",0.790873,0.0343482
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8297872340425532, 0.7975206611570248, 0.820...",0.790852,0.0471267


In [28]:
MLA_breath_25

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8595744680851064, 0.7975206611570248, 0.85,...",0.805674,0.0461046
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8728, 0.8873692679002414, 0.889156626506024...",0.880016,"[0.825531914893617, 0.7892561983471075, 0.8166...",0.795209,0.0420258
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9504, 0.9549477071600966, 0.955020080321285...",0.955213,"[0.7957446808510639, 0.8099173553719008, 0.837...",0.790484,0.0436953
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.851063829787234, 0.7644628099173554, 0.8208...",0.778041,0.0541548


In [29]:
MLA_breath_30

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8638297872340426, 0.7851239669421488, 0.829...",0.802901,0.0486882
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.952, 0.9573612228479486, 0.9550200803212852...",0.955625,"[0.7957446808510639, 0.8140495867768595, 0.837...",0.782119,0.0452174
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8468085106382979, 0.7520661157024794, 0.812...",0.775118,0.051421
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.884, 0.8873692679002414, 0.8811244979919679...",0.88105,"[0.7957446808510639, 0.756198347107438, 0.7958...",0.77511,0.0334781


In [30]:
MLA_breath_35

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 0.999195494770716, 1.0, 1.0, 1.0, 1.0, 1...",0.99992,"[0.8723404255319149, 0.768595041322314, 0.8166...",0.800505,0.0579584
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8425531914893617, 0.7603305785123967, 0.787...",0.775458,0.0523786
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.952, 0.9525341914722446, 0.9534136546184739...",0.955146,"[0.7914893617021277, 0.7768595041322314, 0.783...",0.762412,0.0346969
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8672, 0.8801287208366855, 0.882730923694779...",0.876786,"[0.8127659574468085, 0.7272727272727273, 0.775...",0.761259,0.0481826


In [31]:
MLA_breath_40

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9992, 0.998390989541432, 0.9991967871485944...",0.999436,"[0.8851063829787233, 0.756198347107438, 0.7916...",0.792826,0.0525066
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9488, 0.9501206757843926, 0.953413654618473...",0.952795,"[0.8042553191489362, 0.7644628099173554, 0.795...",0.773193,0.0309817
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8382978723404255, 0.7644628099173554, 0.779...",0.768823,0.0431664
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8648, 0.8849557522123894, 0.872289156626506...",0.867405,"[0.8340425531914893, 0.71900826446281, 0.79583...",0.762994,0.0518921


In [32]:
MLA_breath_45

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9984, 0.999195494770716, 0.9991967871485944...",0.999275,"[0.851063829787234, 0.78099173553719, 0.791666...",0.794615,0.040648
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9512, 0.9525341914722446, 0.955823293172690...",0.954806,"[0.8085106382978723, 0.7975206611570248, 0.770...",0.770629,0.0358797
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.825531914893617, 0.7768595041322314, 0.7708...",0.767907,0.0356545
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8664, 0.8728881737731295, 0.864257028112449...",0.865558,"[0.825531914893617, 0.7231404958677686, 0.7583...",0.751947,0.0336833


In [33]:
MLA_breath_50

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9992, 0.998390989541432, 0.9991967871485944...",0.999031,"[0.8042553191489362, 0.743801652892562, 0.8041...",0.774149,0.0485949
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9504, 0.9501206757843926, 0.952610441767068...",0.953279,"[0.8127659574468085, 0.7603305785123967, 0.770...",0.764194,0.0321082
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8704, 0.8817377312952535, 0.864257028112449...",0.869349,"[0.8085106382978723, 0.7231404958677686, 0.75,...",0.758512,0.0395659
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8042553191489362, 0.743801652892562, 0.7625...",0.755374,0.0418318


### Exp 8

In [35]:
make_legit_sets(breath_dataframe, 'breath', "breath_sets_old/")
make_legit_sets(breath_dataframe_1, 'breath_1', "breath_sets_old/")
make_legit_sets(breath_dataframe_2, 'breath_2', "breath_sets_old/")
make_legit_sets(breath_dataframe_3, 'breath_3', "breath_sets_old/")

In [36]:
MLA_breath = MLA_selection(breath_dataframe, 'breath', "./breathe_set_csv_exp_6/")
MLA_breath_1 = MLA_selection(breath_dataframe_1, 'breath_1', "./breathe_set_csv_exp_6/")
MLA_breath_2 = MLA_selection(breath_dataframe_2, 'breath_2', "./breathe_set_csv_exp_6/")
MLA_breath_3 = MLA_selection(breath_dataframe_3, 'breath_3', "./breathe_set_csv_exp_6/")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[07:51:50] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[07:52:15] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[07:52:30] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[07:52:42] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[07:55:18] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[07:55:22] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[07:55:25] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[07:55:27] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [37]:
MLA_breath

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.875, 0.7216494845360825, 0.712707182320442,...",0.741803,0.0725535
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8641304347826086, 0.6855670103092784, 0.723...",0.740507,0.0691253
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.974739970282318, 0.9819004524886877, 0.9866...",0.977758,"[0.8478260869565217, 0.7216494845360825, 0.668...",0.723863,0.0686187
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8870728083209509, 0.8868778280542986, 0.915...",0.900595,"[0.7119565217391305, 0.6907216494845361, 0.585...",0.689,0.0769933


In [38]:
MLA_breath_1

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9331352154531947, 0.9622926093514329, 0.943...",0.938553,"[0.7228260869565217, 0.6494845360824743, 0.635...",0.693654,0.0638422
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7608695652173914, 0.6391752577319587, 0.651...",0.693614,0.0928434
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.717391304347826, 0.6597938144329897, 0.6519...",0.690205,0.0924723
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8335809806835067, 0.8401206636500754, 0.855...",0.839996,"[0.6902173913043478, 0.6237113402061856, 0.596...",0.657478,0.0658396


In [39]:
MLA_breath_2

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8641304347826086, 0.788659793814433, 0.7513...",0.768649,0.0709488
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8695652173913043, 0.7371134020618557, 0.762...",0.766114,0.0653371
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9643387815750372, 0.9728506787330317, 0.970...",0.970323,"[0.8532608695652174, 0.711340206185567, 0.7348...",0.740438,0.0718545
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8692421991084696, 0.8748114630467572, 0.915...",0.893771,"[0.8097826086956522, 0.6958762886597938, 0.674...",0.711404,0.0773889


In [40]:
MLA_breath_3

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.75, 0.6752577319587629, 0.7513812154696132,...",0.722557,0.0521579
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9301634472511144, 0.9411764705882353, 0.952...",0.940936,"[0.7934782608695652, 0.7268041237113402, 0.668...",0.718348,0.0507701
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7445652173913043, 0.6752577319587629, 0.723...",0.714554,0.0557361
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8320950965824666, 0.8627450980392157, 0.868...",0.853296,"[0.7228260869565217, 0.711340206185567, 0.6740...",0.67073,0.0496967


In [48]:
make_legit_sets(Breath_dataframe_0, 'breath_0', 'breath_sets_old')
make_legit_sets(Breath_dataframe_5, 'breath_5', 'breath_sets_old')
make_legit_sets(Breath_dataframe_10, 'breath_10', 'breath_sets_old')
make_legit_sets(Breath_dataframe_15, 'breath_15', 'breath_sets_old')
make_legit_sets(Breath_dataframe_20, 'breath_20', 'breath_sets_old')
make_legit_sets(Breath_dataframe_25, 'breath_25', 'breath_sets_old')
make_legit_sets(Breath_dataframe_30, 'breath_30', 'breath_sets_old')
make_legit_sets(Breath_dataframe_35, 'breath_35', 'breath_sets_old')
make_legit_sets(Breath_dataframe_40, 'breath_40', 'breath_sets_old')
make_legit_sets(Breath_dataframe_45, 'breath_45', 'breath_sets_old')
make_legit_sets(Breath_dataframe_50, 'breath_50', 'breath_sets_old')

In [50]:
MLA_breath_0 = MLA_selection(Breath_dataframe_0, 'breath_0', './breathe_set_csv_exp_6/') 
MLA_breath_5 = MLA_selection(Breath_dataframe_5, 'breath_5', './breathe_set_csv_exp_6/') 
MLA_breath_10 = MLA_selection(Breath_dataframe_10, 'breath_10', './breathe_set_csv_exp_6/') 
MLA_breath_15 = MLA_selection(Breath_dataframe_15, 'breath_15', './breathe_set_csv_exp_6/') 
MLA_breath_20 = MLA_selection(Breath_dataframe_20, 'breath_20', './breathe_set_csv_exp_6/') 
MLA_breath_25 = MLA_selection(Breath_dataframe_25, 'breath_25', './breathe_set_csv_exp_6/') 
MLA_breath_30 = MLA_selection(Breath_dataframe_30, 'breath_30', './breathe_set_csv_exp_6/') 
MLA_breath_35 = MLA_selection(Breath_dataframe_35, 'breath_35', './breathe_set_csv_exp_6/') 
MLA_breath_40 = MLA_selection(Breath_dataframe_40, 'breath_40', './breathe_set_csv_exp_6/') 
MLA_breath_45 = MLA_selection(Breath_dataframe_45, 'breath_45', './breathe_set_csv_exp_6/') 
MLA_breath_50 = MLA_selection(Breath_dataframe_50, 'breath_50', './breathe_set_csv_exp_6/') 

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[08:04:06] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:04:06] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:04:06] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:04:07] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[08:06:08] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:06:08] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:06:09] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:06:09] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [51]:
MLA_breath_0

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8260869565217391, 0.7216494845360825, 0.745...",0.737362,0.0825438
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9598811292719168, 0.9653092006033183, 0.957...",0.952993,"[0.7717391304347826, 0.7010309278350515, 0.651...",0.732801,0.0669479
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7934782608695652, 0.6752577319587629, 0.729...",0.723366,0.0767976
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8603268945022289, 0.8672699849170438, 0.894...",0.870068,"[0.6956521739130435, 0.6649484536082474, 0.613...",0.688456,0.0639412


In [52]:
MLA_breath_5

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9658246656760773, 0.9683257918552036, 0.961...",0.956852,"[0.8206521739130435, 0.7164948453608248, 0.651...",0.738798,0.0643573
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.842391304347826, 0.7319587628865979, 0.7348...",0.731268,0.0817446
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7989130434782609, 0.6804123711340206, 0.734...",0.726556,0.0685892
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8707280832095097, 0.8778280542986425, 0.918...",0.889142,"[0.7228260869565217, 0.6855670103092784, 0.607...",0.702891,0.0633089


In [53]:
MLA_breath_10

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9702823179791976, 0.971342383107089, 0.9733...",0.967014,"[0.8260869565217391, 0.7216494845360825, 0.690...",0.754768,0.0617191
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.842391304347826, 0.7731958762886598, 0.7348...",0.748394,0.0882188
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8811292719167905, 0.8929110105580694, 0.924...",0.904387,"[0.8043478260869565, 0.7474226804123711, 0.646...",0.745336,0.0643997
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8152173913043478, 0.7371134020618557, 0.756...",0.742462,0.0642041


In [54]:
MLA_breath_15

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9702823179791976, 0.9743589743589743, 0.977...",0.969248,"[0.8478260869565217, 0.7577319587628866, 0.729...",0.758534,0.0632446
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8532608695652174, 0.788659793814433, 0.7237...",0.75525,0.0745084
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8260869565217391, 0.7216494845360825, 0.745...",0.737199,0.0709476
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8826151560178306, 0.8944193061840121, 0.914...",0.902278,"[0.7880434782608695, 0.711340206185567, 0.6629...",0.727437,0.0700392


In [55]:
MLA_breath_20

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8478260869565217, 0.788659793814433, 0.7348...",0.766237,0.0658716
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9777117384843982, 0.971342383107089, 0.9792...",0.971039,"[0.8478260869565217, 0.7371134020618557, 0.729...",0.751719,0.071804
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8532608695652174, 0.7010309278350515, 0.729...",0.746433,0.0738964
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8781575037147102, 0.8989441930618401, 0.921...",0.901863,"[0.7880434782608695, 0.6855670103092784, 0.679...",0.723678,0.0794505


In [56]:
MLA_breath_25

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8478260869565217, 0.7731958762886598, 0.756...",0.762712,0.0584231
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8532608695652174, 0.7164948453608248, 0.756...",0.758042,0.0626374
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9643387815750372, 0.9728506787330317, 0.970...",0.970323,"[0.8532608695652174, 0.711340206185567, 0.7348...",0.740438,0.0718545
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8707280832095097, 0.8778280542986425, 0.917...",0.893776,"[0.8043478260869565, 0.6958762886597938, 0.674...",0.709821,0.077047


In [57]:
MLA_breath_30

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8695652173913043, 0.7268041237113402, 0.784...",0.774117,0.0643719
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.842391304347826, 0.7525773195876289, 0.7403...",0.749133,0.0639928
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9687964338781575, 0.9683257918552036, 0.970...",0.969417,"[0.8532608695652174, 0.6804123711340206, 0.729...",0.729471,0.0709451
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8870728083209509, 0.8853695324283559, 0.917...",0.898271,"[0.8152173913043478, 0.6752577319587629, 0.685...",0.70237,0.0790634


In [58]:
MLA_breath_35

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8315217391304348, 0.6907216494845361, 0.729...",0.752261,0.0595969
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8260869565217391, 0.7319587628865979, 0.767...",0.748704,0.0630833
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.962852897473997, 0.9638009049773756, 0.9600...",0.965262,"[0.8695652173913043, 0.6958762886597938, 0.723...",0.72203,0.0715846
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8796433878157504, 0.8868778280542986, 0.906...",0.894394,"[0.7880434782608695, 0.6597938144329897, 0.657...",0.679365,0.0704867


In [59]:
MLA_breath_40

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7771739130434783, 0.6649484536082474, 0.745...",0.742323,0.0668814
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8206521739130435, 0.7164948453608248, 0.740...",0.739663,0.0615208
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9613670133729569, 0.9653092006033183, 0.957...",0.962888,"[0.8315217391304348, 0.711340206185567, 0.7016...",0.709868,0.0601782
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8454680534918276, 0.8657616892911011, 0.890...",0.877137,"[0.7336956521739131, 0.6597938144329897, 0.646...",0.661563,0.0605537


In [60]:
MLA_breath_45

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8369565217391305, 0.6958762886597938, 0.751...",0.729557,0.0658449
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8315217391304348, 0.6701030927835051, 0.734...",0.722865,0.0625562
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9598811292719168, 0.9577677224736049, 0.958...",0.962128,"[0.8369565217391305, 0.7216494845360825, 0.696...",0.712341,0.0632707
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.826151560178306, 0.8476621417797888, 0.8757...",0.863412,"[0.7010869565217391, 0.6443298969072165, 0.640...",0.6319,0.068103


In [61]:
MLA_breath_50

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8315217391304348, 0.6701030927835051, 0.707...",0.718925,0.0655402
3,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.7717391304347826, 0.6597938144329897, 0.729...",0.718689,0.0534046
1,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9613670133729569, 0.9592760180995475, 0.958...",0.960336,"[0.8097826086956522, 0.7010309278350515, 0.662...",0.699723,0.054432
2,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8395245170876672, 0.861236802413273, 0.8801...",0.864306,"[0.7119565217391305, 0.6649484536082474, 0.624...",0.654585,0.0767501
